<a href="https://colab.research.google.com/github/jm-tan-jm/web_scrap_agent_info/blob/failed/get_Agent_Info_mudah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## get agent profile links only, as the phone number are hidden

In [ ]:
!pip install selenium webdriver_manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.3 MB/s eta 0:00:00


In [ ]:
# function 1 - get the item's link from the mudah url with required parameters
## Limitation - do one page only each time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

def get_item_links(core_url="https://www.mudah.my", region=None, category=None, page=None):
    if region is None or category is None:
        raise ValueError("Please provide valid values for 'region' or 'category'")

    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options)

    try:
        if page == 1:
            url = rf"{core_url}/{region}/{category}"
        else:
            url = rf"{core_url}/{region}/{category}?o={page}"

        driver.get(url)

        # Wait for the item container to be present
        WebDriverWait(driver, 10)

        item_links = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-gxMtzJ dezEbJ')]/a")
        all_item_links = [item_link.get_attribute('href') for item_link in item_links]

        return all_item_links

    except Exception as e:
        print(f"Error: {e}")
        return []

    finally:
        driver.quit()

In [ ]:
# function 2 - get the agent profile's link from the url
## Limitation - can only handle one link each time, need to iterate the function for each link
def get_agent_link(item_link):
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options)

    try:
        driver.get(item_link)
        WebDriverWait(driver, 10)

        agent_links = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-fgfRvd iuuSSK')]/a")
        all_agent_link = [agent_link.get_attribute('href') for agent_link in agent_links]

        return all_agent_link

    except Exception as e:
        print(f"Error: {e}")
        return []

    finally:
        driver.quit()

    return agent_info

In [ ]:
# to run above two functions and convert the result to Excel
import pandas as pd

# to get multiple agents' profile
item_links = []

# loop for multiple pages
for page_number in range(1, 3):  # Scrapes pages 1 to 2
    # function 1
    item_link = get_item_links(core_url="https://www.mudah.my", region="kuala-lumpur", category="properties-for-rent", page=None)
    item_links.extend(item_link)

agent_links = []
# to get agent profile link from each links above
for item_link in item_links:
    # function 2
    agent_link = get_agent_link(item_link)
    agent_links.extend(agent_link)

# Remove duplicates
unique_agent_links = list(set(agent_links))

# Convert the list to a DataFrame
df = pd.DataFrame(unique_agent_links, columns=['Agent Links'])

print(df)

with open('output.txt', 'w') as f:
  print(df, file=f)

Empty DataFrame
Columns: [Agent Links]
Index: []
